In [1]:
from scipy import stats
import pandas as pd
import numpy as np
import sklearn.preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import math
from functools import reduce

Unhash any command to run.
Read in file. Drop unnecessary columns. Save copy where drop empty rows.

In [2]:
# Read the Excel file
df = pd.read_excel('Parts and Dimesions.xlsx')
#print(df.head)

# Do not use the 'Operator' or 'Item_No' column
df = df.drop(columns=['Item_No', 'Operator'])
#print(df.head)

# Remove rows with missing cells
df_nomissing = df.dropna()

# Print the updated DataFrame
#print(df_nomissing)

Outlier checks
check for anything outside of 2 standard deviations over global values

In [3]:
def zscoreGlobalOutlier(df, stdev_threshold):
    # Calculate the z-score for each column
    z_scores = stats.zscore(df)

    # Identify outliers
    outliers = z_scores > stdev_threshold

    # Remove outliers from the DataFrame
    df_global_outlier_removed = df[~outliers.any(axis=1)]

    return df_global_outlier_removed

In [4]:
df_outlierthreshold = df_nomissing.copy()
df_noGlobalOutlier = zscoreGlobalOutlier(df_outlierthreshold, 2)
print(df_noGlobalOutlier)

     Length  Width  Height
0    102.67  49.53   19.69
1    102.50  51.42   19.63
2     95.37  52.25   21.51
3     94.77  49.24   18.60
4    104.26  47.90   19.46
..      ...    ...     ...
494   98.34  49.03   19.30
495  101.24  49.03   20.96
496   98.37  52.12   19.68
498   94.16  48.39   21.60
499  102.35  51.24   21.47

[421 rows x 3 columns]


Outlier check
check for anything outside of 2 standard deviations over sliding window

Outlier check
Isolation Forest Algorithm

Normalization methods
MinMax

Normalization methods
z-score normalization

In [5]:
def zScoreNormalized(df):
    z_scores = (df - df.mean()) / df.std()
    df_normalized = (z_scores - z_scores.min()) / (z_scores.max() - z_scores.min())
    return df_normalized

In [6]:
df_zscoreNorm = df_noGlobalOutlier.copy()
df_zScorNormalized = zScoreNormalized(df_zscoreNorm)
print(df_zScorNormalized)

       Length     Width    Height
0    0.751116  0.607335  0.501080
1    0.742687  0.776386  0.488121
2    0.389192  0.850626  0.894168
3    0.359445  0.581395  0.265659
4    0.829945  0.461538  0.451404
..        ...       ...       ...
494  0.536440  0.562612  0.416847
495  0.680218  0.562612  0.775378
496  0.537928  0.838998  0.498920
498  0.329202  0.505367  0.913607
499  0.735250  0.760286  0.885529

[421 rows x 3 columns]


Normalization methods
L2 Normalization

In [7]:
def l2Normalization(df):
    for column in df.columns:
        reshaped_data = df[column]
        normalized_column = reshaped_data / np.linalg.norm(reshaped_data)
        df[column] = normalized_column
    return df

In [8]:
df_l2norm = df_noGlobalOutlier.copy()
df_L2Normalized = l2Normalization(df_l2norm)
print(df_L2Normalized)

       Length     Width    Height
0    0.050505  0.048480  0.047404
1    0.050422  0.050330  0.047259
2    0.046914  0.051142  0.051785
3    0.046619  0.048196  0.044780
4    0.051287  0.046884  0.046850
..        ...       ...       ...
494  0.048375  0.047990  0.046465
495  0.049802  0.047990  0.050461
496  0.048390  0.051015  0.047380
498  0.046319  0.047364  0.052002
499  0.050348  0.050153  0.051689

[421 rows x 3 columns]


Make method to randomly remove data

In [27]:
'''def randomly_remove_indices(df, percentage):
    # Calculate the number of indices to remove
    num_indices = int(len(df) * percentage/100)

    # split each column into its own df
    dfL = df['Length']
    dfW = df['Width']
    dfH = df['Height']

    # Generate random indices to remove for each colunm
    indices_to_removeL = np.random.choice(dfL.index, size=num_indices, replace=False)
    indices_to_removeW = np.random.choice(dfW.index, size=num_indices, replace=False)
    indices_to_removeH = np.random.choice(dfH.index, size=num_indices, replace=False)

    # Save the removed indices as groundTruth
    groundTruthL = dfL.loc[indices_to_removeL].copy()
    groundTruthW = dfW.loc[indices_to_removeW].copy()
    groundTruthH = dfH.loc[indices_to_removeH].copy()

    # Remove the indices from individual columns
    newL = dfL.copy()
    newL.loc[indices_to_removeL] = np.nan
    newW = dfW.copy()
    newW.loc[indices_to_removeW] = np.nan
    newH = dfH.copy()
    newH.loc[indices_to_removeH] = np.nan

    def merge_dfs(df1, df2):
        return pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
    
    df_merged = reduce(merge_dfs, [newL, newW, newH])
    df_mergedGT = reduce(merge_dfs, [groundTruthL, groundTruthW, groundTruthH])

    return df_merged, df_mergedGT'''

"def randomly_remove_indices(df, percentage):\n    # Calculate the number of indices to remove\n    num_indices = int(len(df) * percentage/100)\n\n    # split each column into its own df\n    dfL = df['Length']\n    dfW = df['Width']\n    dfH = df['Height']\n\n    # Generate random indices to remove for each colunm\n    indices_to_removeL = np.random.choice(dfL.index, size=num_indices, replace=False)\n    indices_to_removeW = np.random.choice(dfW.index, size=num_indices, replace=False)\n    indices_to_removeH = np.random.choice(dfH.index, size=num_indices, replace=False)\n\n    # Save the removed indices as groundTruth\n    groundTruthL = dfL.loc[indices_to_removeL].copy()\n    groundTruthW = dfW.loc[indices_to_removeW].copy()\n    groundTruthH = dfH.loc[indices_to_removeH].copy()\n\n    # Remove the indices from individual columns\n    newL = dfL.copy()\n    newL.loc[indices_to_removeL] = np.nan\n    newW = dfW.copy()\n    newW.loc[indices_to_removeW] = np.nan\n    newH = dfH.copy()\n

In [28]:
'''X = randomly_remove_indices(df_L2Normalized, 10)
#X[0]
X[1]'''

'X = randomly_remove_indices(df_L2Normalized, 10)\n#X[0]\nX[1]'

In [23]:
def randomly_remove_indices1(df, percentage):
    indices_to_remove = {}
    groundTruth = {}
    new_df = {}
    num_indices = int(len(df) * percentage/100)
    
    for column in df.columns:
        indices_to_remove[column] = np.random.choice(df[column].index, size=num_indices, replace=False)
        groundTruth[column] = df[column].loc[indices_to_remove[column]].copy()
        new_df[column] = df[column].copy()
        new_df[column].loc[indices_to_remove[column]] = np.nan

    # Merge the new DataFrames
    df_merged = pd.concat(new_df.values(), axis=1)
    df_mergedGT = pd.concat(groundTruth.values(), axis=1)

    return df_merged, df_mergedGT

In [25]:
X1 = randomly_remove_indices1(df_L2Normalized, 10)
X1[1]

,Length,Width,Height
0,NaN,0.048480,NaN
7,NaN,NaN,0.050630
11,NaN,0.047237,NaN
13,NaN,NaN,0.048872
15,0.046983,NaN,NaN
...,...,...,...
484,NaN,NaN,0.045694
491,0.049408,NaN,0.047476
495,0.049802,NaN,NaN
498,NaN,NaN,0.052002


Data Imputation Methods
Neural Network

Data Imputation Methods
MICE

Data Imputation Methods
Bayesian

Scoring Methods

Data Visualization